In [47]:
import pandas as pd
import os
import duckdb

In [54]:
from dotenv import load_dotenv, find_dotenv
load_dotenv('/workspaces/patient_conditions_weather/src/.env')

True

In [55]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/workspaces/patient_conditions_weather/airflow/credentials/google_credentials.json'
emr_path = f"gs://{os.getenv('EMR_RAW_BUCKET')}/{os.getenv('EMR_RAW_PATH')}"
encounters_path = emr_path +'/encounters.csv'
organizations_path = emr_path + '/organizations.csv'
conditions_path = emr_path + '/conditions.csv'
emr_path

'gs://patient-weather-data/emr-data-raw'

In [56]:
encounters = pd.read_csv(encounters_path, delimiter=",")
conditions = pd.read_csv(conditions_path, delimiter=",")
organizations = pd.read_csv(organizations_path, delimiter=",")
# Create a DuckDB connection
con = duckdb.connect()
con.register('encounters', encounters)
con.register('conditions', conditions)
con.register('organizations', organizations)




In [37]:
organizations

,Id,NAME,ADDRESS,CITY,STATE,ZIP,LAT,LON,PHONE,REVENUE,UTILIZATION
0,ef58ea08-d883-3957-8300-150554edc8fb,HEALTHALLIANCE HOSPITALS INC,60 HOSPITAL ROAD,LEOMINSTER,MA,01453,42.520838,-71.770876,9784662000,0.0,1214
1,69176529-fd1f-3b3f-abce-a0a3626769eb,MOUNT AUBURN HOSPITAL,330 MOUNT AUBURN STREET,CAMBRIDGE,MA,02138,42.375967,-71.118275,6174923500,0.0,2877
2,5e765f2b-e908-3888-9fc7-df2cb87beb58,STURDY MEMORIAL HOSPITAL,211 PARK STREET,ATTLEBORO,MA,02703,41.931653,-71.294503,5082225200,0.0,2365
3,f1fbcbfb-fcfa-3bd2-b7f4-df20f1b3c3a4,LAWRENCE GENERAL HOSPITAL,ONE GENERAL STREET,LAWRENCE,MA,01842,42.700273,-71.161357,9786834000,0.0,976
4,e002090d-4e92-300e-b41e-7d1f21dee4c6,CAMBRIDGE HEALTH ALLIANCE,1493 CAMBRIDGE STREET,CAMBRIDGE,MA,02138,42.375967,-71.118275,6176652300,0.0,2706
...,...,...,...,...,...,...,...,...,...,...,...
1122,700056a6-3aea-3efd-8423-a9acbd38a5ea,ROUTE 44 MEDICAL WALK-IN,106 UNITED STATES HIGHWAY 44,RAYNHAM,MA,2767,41.905194,-71.060999,508-824-1476,0.0,40
1123,308b5efa-a9c2-3728-b1ff-d29c5f060731,FALLON CLINIC URGENT CARE,630 PLANTATION STREET,WORCESTER,MA,1605,42.292559,-71.760317,508-595-2505,0.0,152
1124,bbf934c6-faad-3483-a1d2-d2ecb693a0e2,FALMOUTH WALK IN MEDICAL CENTER,309 TEATICKET HIGHWAY,TEATICKET,MA,2536,41.568344,-70.594006,508-540-6790,0.0,11
1125,90cf148c-69ed-3d33-aa86-b509c6b0b25f,PROMPT CARE WALK-IN CLINIC,170 GOVERNORS AVENUE,MEDFORD,MA,2155,42.425909,-71.111365,781-306-6180,0.0,72


In [57]:
query = """
with events as (
    select 
        extract('year' FROM  CAST(e.stop AS TIMESTAMP))  as year,
        o.name, o.city,
        c.code,
        c.description,
        round(o.LAT,1) as LAT,
        round(o.LON,1) as LON
    from encounters e
    inner join organizations o on e.ORGANIZATION = o.id
    inner join conditions c on e.id = c.ENCOUNTER
)
select distinct
    year,
    -- name,city,
    LAT as latitude, 
    LON as longitude
from events
where year > 1980
order by year, lat,lon, name,city
"""
years_coordinates = con.execute(query).fetchdf()
#full count conditions join encounters = 38094
years_coordinates

,year,latitude,longitude
0,1981,41.6,-71.1
1,1981,41.6,-71.0
2,1981,41.6,-70.9
3,1981,41.6,-70.6
4,1981,41.6,-70.5
...,...,...,...
4582,2021,42.7,-70.8
4583,2021,42.8,-71.1
4584,2021,42.8,-71.0
4585,2021,42.8,-70.9


In [58]:


event_years_coordinates_filepath = f"gs://{os.getenv('WEATHER_BUCKET')}/{os.getenv('EVENT_YEARS_COORDINATES_FILEPATH')}"
print(event_years_coordinates_filepath)
event_years_coordinates = pd.read_csv(event_years_coordinates_filepath, delimiter=",")
event_years_coordinates



gs://patient-weather-data/weather-data-raw/event_years_coordinates.csv


,year,latitude,longitude
0,1980,41.6,-71.1
1,1980,41.6,-71.0
2,1980,41.6,-70.9
3,1980,41.6,-70.6
4,1980,41.6,-70.5
...,...,...,...
4660,2021,42.7,-70.8
4661,2021,42.8,-71.1
4662,2021,42.8,-71.0
4663,2021,42.8,-70.9


In [82]:
import random 



columns = random.sample(['latitude', 'longitude', 'year'], k=3)
ascending = [random.choice([True, False]) for _ in columns]
event_years_coordinates.sort_values(by=columns, ascending=ascending, inplace=True)
event_years_coordinates

,year,latitude,longitude
185,1982,41.8,-70.0
450,1985,41.8,-70.0
548,1986,41.8,-70.0
1007,1991,41.8,-70.0
1100,1992,41.8,-70.0
...,...,...,...
3871,2016,42.1,-73.4
4006,2017,42.1,-73.4
4143,2018,42.1,-73.4
4284,2019,42.1,-73.4


In [68]:

event_years_coordinates[
 (event_years_coordinates['latitude'] == 41.6) 
                    & (event_years_coordinates['longitude'] == -71.1)]

,year,latitude,longitude
0,1980,41.6,-71.1
78,1981,41.6,-71.1
342,1984,41.6,-71.1
533,1986,41.6,-71.1
625,1987,41.6,-71.1
899,1990,41.6,-71.1
1086,1992,41.6,-71.1
1185,1993,41.6,-71.1
1280,1994,41.6,-71.1
1383,1995,41.6,-71.1


In [66]:

event_years_coordinates[
(event_years_coordinates['year'].isin([1979,  1982, 1983]))
 & (event_years_coordinates['latitude'] == 41.6) 
                    & (event_years_coordinates['longitude'] == -71.1)].empty

True